# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [5]:
# path to the model
dir_model = "../model/xyz_DNN/"

# you can try different model
# dir_model = "../Model/xyz_hand/"
# dir_model = "../Model/xy/"
# dir_model = "../Model/xy_hand/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]

# 2. Setup OpenCV and MediaPipe to Predict

In [43]:
dir_video_test = ('../../data/ready/perfect/reguler/5.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

i_momentum = 0
i_non_momentum = 0
#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()
        
        # recolor feed
        # image = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            X = pd.DataFrame([row])
            momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xyz_hand
            # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy_hand
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            # momentum_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            # momentum_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            print(momentum_class, momentum_prob)

            if momentum_class == 0 and momentum_prob >= 0.9 :
                current_stage = 'Nice!'
                i_non_momentum+=1
            elif momentum_class == 1 and momentum_prob >= 0.9 :
                current_stage = 'Use momentum!'
                i_momentum+=1

            # get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            #Display class
            cv2.putText(image, 'Class', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            #Display Prob
            cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(momentum_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
            
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1 1.0
'NoneType' object has no attribute 'landmark'
1 1.0
1 1.0
1 1.0
1 1.0
'NoneType' object has no attribute 'landmark'
0 0.99883294
0 0.9971121
0 0.99219877
0 0.9909069
0 0.9990825
0 0.9998025
0 0.99989074
0 0.9998012
0 0.9997335
0 0.9999459
0 0.9998774
0 0.9998454
0 0.9996335
0 0.9996957
0 0.63986415
1 0.5508854
1 0.7557831
1 0.69534856
1 0.77486193
1 0.88049865
1 0.9494375
1 0.97381717
1 0.9629841
1 0.95204824
1 0.951311
1 0.91490686
1 0.92614466
1 0.66891706
0 0.5671182
0 0.929233
0 0.9938806
0 0.9995013
0 0.99985266
0 0.9998124
0 0.99872017
0 0.99847263
0 0.99642664
0 0.9998216
0 0.9999493
0 0.9998224
0 0.99964106
0 0.99893034
0 0.99568903
0 0.9873416
0 0.99546707
0 0.99871093
0 0.9995256
0 0.9995213
0 0.9996237
0 0.99986887
0 0.9998782
0 0.9997995
0 0.9999269
0 0.9998138
0 0.9996679
0 0.9995759
0 0.99923784
0 0.83074695
1 0.8029502
1 0.7869902
1 0.93195593
1 0.89106596
1 0.8892549
1 0.89923066
1 0.91339964
1 0.87464976
1 0.8140688
1 0.7870503
1 0.86935073
1 0.74005854
1 0.64696

In [39]:
print(i_momentum, i_non_momentum)

190 27
